# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [31]:
import azureml.core
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling, BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform, quniform
from azureml.core import Environment, ScriptRunConfig
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.experiment import Experiment
from azureml.data.datapath import DataPath
import os

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.56.0


## Dataset

### Overview
In this project, we are forcusing on deploying the model endpoint to predict the Hearth failure, we used the both AutoML and Hyperparameter turning to train the data and find the best model among them. We will use it to deploy the model endpoint and consume it to predict patient's survival based on the collected patient data.

We analyzed a dataset containing the medical records of 299 heart failure patients collected at the Faisalabad Institute of Cardiology and at the Allied Hospital in Faisalabad. The patients consisted of 105 woman and 194 men, and their ages between 40 and 95 year old.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'random-forest-heart-failure'

experiment=Experiment(ws, experiment_name)

In [32]:
found = False
key = "Heart Failure Dataset"
description_text = "Heart Failure Dataset for Udacity Capstone"

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]

if not found:
    datastore = ws.get_default_datastore()
    datastore.upload_files(files = ['../dataset/heart_failure_clinical_records_dataset.csv'],  target_path='data/', overwrite=True, show_progress=True)
    example_data = 'data/heart_failure_clinical_records_dataset.csv'
    datastore_path =[
        DataPath(datastore, example_data)
    ]
    dataset = Dataset.Tabular.from_delimited_files(path=datastore_path)
    dataset = dataset.register(workspace=ws,name=key,description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,time
count,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000
mean,60.833893,581.839465,38.083612,263358.029264,1.39388,136.625418,130.260870
std,11.894809,970.287881,11.834841,97804.236869,1.03451,4.412477,77.614208
min,40.000000,23.000000,14.000000,25100.000000,0.50000,113.000000,4.000000
25%,51.000000,116.500000,30.000000,212500.000000,0.90000,134.000000,73.000000
50%,60.000000,250.000000,38.000000,262000.000000,1.10000,137.000000,115.000000
75%,70.000000,582.000000,45.000000,303500.000000,1.40000,140.000000,203.000000
max,95.000000,7861.000000,80.000000,850000.000000,9.40000,148.000000,285.000000


In [39]:
dataset.take(5).to_pandas_dataframe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,False,582,False,20,True,265000.00,1.9,130,True,False,4,True
1,55.0,False,7861,False,38,False,263358.03,1.1,136,True,False,6,True
2,65.0,False,146,False,20,False,162000.00,1.3,129,True,True,7,True
3,50.0,True,111,False,20,False,210000.00,1.9,137,True,False,7,True
4,65.0,True,160,True,20,False,327000.00,2.7,116,False,False,8,True


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "hyper-parameter-turning"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_A2m_v2',# for GPU, use "STANDARD_NC6" STANDARD_D2_V2 Standard_A2m_v2
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [48]:
print(str(dataset.as_named_input('heartfailure')))

In [49]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(
    evaluation_interval=2,
    slack_factor=0.1
)

#TODO: Create the different params that you will be using during training
param_sampling = BayesianParameterSampling(
    {
        '--n_estimators': choice(100,200,300,400),
        '--max_depth': quniform(10,200,1),
        '--min_samples_leaf': quniform(1,15,1),
        '--criterion': choice("gini", "entropy", "log_loss")
    }
)

# param_sampling = RandomParameterSampling(
#     {
#         '--n_estimators': choice(100,200,300,400),
#         '--max_depth': choice(3, 5, 10, 20),
#         '--min_samples_leaf': choice(5, 10, 20, 50, 100),
#         '--criterion': choice("gini", "entropy", "log_loss")
#     }
# )

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='env.yml')


#TODO: Create your estimator and hyperdrive config
src = ScriptRunConfig(
    source_directory="./",
    compute_target=compute_target,
    script="train.py",
    arguments=['--input-data', dataset.as_named_input('heartfailure')],
    environment=sklearn_env
)

hyperdrive_run_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=80,
    max_concurrent_runs=4
)

In [50]:
#TODO: Submit your experiment
run_exp = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [52]:
run_exp.wait_for_completion(show_output=True)

RunId: HD_a1b1510f-e316-4d0b-8eeb-71abae1d9577
Web View: https://ml.azure.com/runs/HD_a1b1510f-e316-4d0b-8eeb-71abae1d9577?wsid=/subscriptions/eab0c039-c811-4365-a933-8367a6b19fe6/resourcegroups/azureml/workspaces/test123&tid=61ba833b-9b71-480d-bd38-3403f22640d2

Streaming azureml-logs/hyperdrive.txt

[2024-07-18T13:31:11.485478][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-07-18T13:31:11.9253086Z][SCHEDULER][INFO]Scheduling job, id='HD_a1b1510f-e316-4d0b-8eeb-71abae1d9577_0' 
[2024-07-18T13:31:12.0440694Z][SCHEDULER][INFO]Scheduling job, id='HD_a1b1510f-e316-4d0b-8eeb-71abae1d9577_1' 
[2024-07-18T13:31:12.1931496Z][SCHEDULER][INFO]Scheduling job, id='HD_a1b1510f-e316-4d0b-8eeb-71abae1d9577_2' 
[2024-07-18T13:31:12.2766514Z][SCHEDULER][INFO]Scheduling job, id='HD_a1b1510f-e316-4d0b-8eeb-71abae1d9577_3' 
[2024-07-18T13:31:12.237952][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2024-07-18T13:31:12.58

{'runId': 'HD_a1b1510f-e316-4d0b-8eeb-71abae1d9577',
 'target': 'hyper-parameter-turning',
 'status': 'Completed',
 'startTimeUtc': '2024-07-18T13:31:10.428256Z',
 'endTimeUtc': '2024-07-18T14:14:35.490566Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '690f664e-e6e8-48d5-b6ac-9bc90f644b23',
  'user_agent': 'python/3.9.19 (Linux-5.15.0-1064-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.56.0',
  'space_size': 'infinite_space_size',
  'best_child_run_id': 'HD_a1b1510f-e316-4d0b-8eeb-71abae1d9577_69',
  'score': '0.8380952380952381',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_a1b1510f-e316-4d0b-8eeb-71abae1d9577_69'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
 

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [53]:
# import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = run_exp.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print("---------------------------------------------------------------------------")
print(f'Run Id: {best_run.id}')
print(f'Accuracy: {best_run_metrics["Accuracy"]}')
print(f'N estimators: {best_run_metrics["n_estimators:"]}')
print(f'Max depth: {best_run_metrics["max_depth:"]}')
print(f'Min sample leaf: {best_run_metrics["min_samples_leaf:"]}')
print(f'Criterion: {best_run_metrics["criterion:"]}')
print("---------------------------------------------------------------------------")

---------------------------------------------------------------------------
Run Id: HD_a1b1510f-e316-4d0b-8eeb-71abae1d9577_69
Accuracy: 0.8380952380952381
N estimators: 200
Max depth: 175
Min sample leaf: 5
Criterion: entropy
---------------------------------------------------------------------------


In [54]:
print("the model is saved in the outputs directory")
os.makedirs("./outputs", exist_ok = True)
best_run.download_file("outputs/model.pkl", "outputs/model.pkl")
print("successful")

the model is saved in the outputs directory
successful


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [55]:
original_model = best_run.register_model(
    model_name="randomforest_model", model_path="outputs/model.pkl"
)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [56]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model
from azureml.core.environment import Environment

service_name = 'random-forest-endpoint'
myenv = Environment.from_conda_specification(name="myenv",file_path='env.yml')

# myenv = get_environment_safe(automl_run)
inference_config = InferenceConfig(entry_script='score.py', environment=myenv)

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1.8, 
    memory_gb=2,
    enable_app_insights=True, 
    auth_enabled=True
)

service = Model.deploy(
    workspace=ws,
    name=service_name,
    inference_config=inference_config,
    models=[original_model],
    deployment_config=aci_config,
    overwrite=True
)

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


In [57]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-07-18 14:15:58+00:00 Creating Container Registry if not exists.
2024-07-18 14:15:59+00:00 Use the existing image.
2024-07-18 14:16:00+00:00 Generating deployment configuration.
2024-07-18 14:16:02+00:00 Submitting deployment to compute..
2024-07-18 14:16:13+00:00 Checking the status of deployment random-forest-endpoint..
2024-07-18 14:18:22+00:00 Checking the status of inference endpoint random-forest-endpoint.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, print the logs of the web service and delete the service

In [58]:
print(service.scoring_uri)
print(service.get_keys())

http://726abd0b-9879-42e7-8ef2-012b46a49a73.southeastasia.azurecontainer.io/score
('BnDrKAXSxJhcqYRNYRWLnVCIg2cw3qy7', 'eWCgV24OceJkcMchOWgDwBBsSImbr42N')


In [78]:
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = service.get_keys()[0]

# Two sets of data to score, so we get two results back
data = {
  "data": [
        {
            "age": 70, 
            "anaemia": False, 
            "creatinine_phosphokinase": 92, 
            "diabetes": False, 
            "ejection_fraction": 60, 
            "high_blood_pressure": True, 
            "platelets": 317000, 
            "serum_creatinine": 0.8, 
            "serum_sodium": 140, 
            "sex": False, 
            "smoking": True, 
            "time": 74
        }
    ],
    "method": "predict"
}

# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

{"result": [false]}


In [77]:
logs = service.get_logs()
for line in logs.split('\n'):
    print(line)

2024-07-18T14:17:51,712115048+00:00 - rsyslog/run 
2024-07-18T14:17:51,721257911+00:00 - gunicorn/run 
2024-07-18T14:17:51,722854922+00:00 - nginx/run 
2024-07-18T14:17:51,724149057+00:00 | gunicorn/run | 
2024-07-18T14:17:51,726072107+00:00 | gunicorn/run | ###############################################
2024-07-18T14:17:51,728020413+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-07-18T14:17:51,730215333+00:00 | gunicorn/run | ###############################################
2024-07-18T14:17:51,732440934+00:00 | gunicorn/run | 
2024-07-18T14:17:51,736244884+00:00 | gunicorn/run | 
2024-07-18T14:17:51,742319391+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20240418.v1
2024-07-18T14:17:51,744228454+00:00 | gunicorn/run | 
2024-07-18T14:17:51,745876871+00:00 | gunicorn/run | 
2024-07-18T14:17:51,748873154+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml_b642e377c6ff7effcfe8950f24786eb4/bin:/opt/mi

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

